In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
from imblearn.metrics import sensitivity_specificity_support

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [16]:
def readCSV():
    data = pd.read_csv('data.csv')
    return data

def cleanData(data):
    data['bedrooms'] = data['bedrooms'].round().astype(int)
    data['bathrooms'] = data['bathrooms'].round().astype(int)
    data['price'] = data['price'].round(decimals=2)
    data['statezip'] = data['statezip'].replace("WA 9", "9",regex=True).astype(int)
    missing = data.loc[(data['price'] == 0)].append(data.loc[(data['bathrooms'] == 0)])
    missing_index_list = missing.reset_index()['index'].to_list()
    missing_index_list.sort(reverse = True)
    data = data.drop(missing_index_list,axis=0)
    return data

def trainModel(data):
    features = data.drop(["date","street","country", "city", "waterfront", "view", "condition", "yr_renovated", "sqft_above", "floors"],axis=1)

    y = features['price']
    X = features.drop(['price'],axis=1)


    dt = DecisionTreeRegressor(max_depth=18)
    dt.fit(X, y)
    
    return dt

def getresponse(userDF, dtpred, aggressionLevel):
    data = readCSV()
    data = cleanData(data)
    dtpred = trainModel(data)
    offerEstimate = dtpred.predict(userDF)
    if aggressionLevel == "low":
        offerEstimate = offerEstimate*.95
    elif aggressionLevel == "high":
        offerEstimate = offerEstimate*1.05

    return f"Using the information provided, {offerEstimate} would be a ressonable offer for this property"

In [10]:
data = readCSV()

In [12]:
data = cleanData(data)

data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3,2,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,98133,USA
1,2014-05-02 00:00:00,2384000.0,5,2,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,98119,USA
2,2014-05-02 00:00:00,342000.0,3,2,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,98042,USA
3,2014-05-02 00:00:00,420000.0,3,2,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,98008,USA
4,2014-05-02 00:00:00,550000.0,4,2,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,98052,USA


In [13]:
dtpred = trainModel(data)

In [14]:
    userData = [{'price': 100000, 'bedrooms': 2, 'bathrooms': 1, 'sqft_living': 1000, 'sqft_lot': 4000, 'sqft_basement': 0, 'yr_built': 1980, 'statezip':98056}]
    userDF = pd.DataFrame(userData)

In [17]:
str = getresponse(userDF, dtpred, "high")

print(str)

Using the information provided, [2309895.] would be a ressonable offer for this property


c:\Anaconda3\envs\dev\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- price
Feature names seen at fit time, yet now missing:
- floors

  warnings.warn(message, FutureWarning)
